# Dataset loading

In [ ]:
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer
from hr_research.config import output_path
from datasets import load_dataset, Dataset
from os.path import join
import itertools

In [ ]:
model_checkpoint = "roberta-large"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
tokenizer.model_max_length

In [ ]:
mlm_data_train = join(output_path, "mlm_data_train.jsonl")
mlm_data_val = join(output_path, "mlm_data_val.jsonl")

In [ ]:
TRAIN_LEN = 22236629
VAL_LEN = 2000
# found experimentally. 
# all datapoints are contatenated during training, 
# therefore each sequence of len 512 is actually more than one datapoint
AVG_DATAPOINT_LEN = 64
ACTUAL_TRAIN_LEN = TRAIN_LEN // (512 // AVG_DATAPOINT_LEN)
ACTUAL_VAL_LEN = VAL_LEN // (512 // AVG_DATAPOINT_LEN)

BATCH_SIZE = 1
NUM_STEPS_PER_EPOCH = ACTUAL_TRAIN_LEN // BATCH_SIZE
EVALUATION_STEPS = (ACTUAL_VAL_LEN * 19) // BATCH_SIZE

In [ ]:
mlm_dataset = load_dataset("json", data_files={
    'train': mlm_data_train, 
    'validation': mlm_data_val}, 
    streaming=True)

mlm_dataset

In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = mlm_dataset.map(
    tokenize_function, batched=True, remove_columns=["text"]
)
tokenized_datasets

In [ ]:
tokenizer.decode(next(iter(tokenized_datasets["train"]))["input_ids"])

In [ ]:
chunk_size = 512

In [ ]:
tokenized_samples = list(t["input_ids"] for t in itertools.islice(tokenized_datasets["train"], 3))

for idx, sample in enumerate(tokenized_samples):
    print(f"'>>> Resume {idx} length: {len(sample)}'")

In [ ]:
tokenized_samples = list(t["input_ids"] for t in itertools.islice(tokenized_datasets["train"], 3))

lens = 0
for idx, sample in enumerate(tokenized_samples):
    lens += len(sample)

In [ ]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

In [ ]:
tokenizer.decode(next(iter(lm_datasets["train"]))["input_ids"])

# Model training

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
samples = list(itertools.islice(lm_datasets["train"], 1))
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

In [ ]:
import collections
import numpy as np

from transformers.data.data_collator import default_data_collator

wwm_probability = 0.2

def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        print(mapping)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [ ]:
samples = list(itertools.islice(lm_datasets["train"], 1))
samples[0]['word_ids']
batch = whole_word_masking_data_collator(samples)

In [ ]:
samples = list(itertools.islice(lm_datasets["train"], 1))
print(f"\n'<<< {tokenizer.decode(samples[0]['input_ids'])}")

batch = whole_word_masking_data_collator(samples)
for i, chunk in enumerate(batch["input_ids"]):
    print(f"\n'>>> {tokenizer.decode(chunk)}'")
    

In [ ]:
samples = list(itertools.islice(lm_datasets["train"], 1))
tokenizer.decode(samples[0]['input_ids'])

In [ ]:
output_dir = './output'

In [ ]:
from transformers import TrainingArguments, Trainer, pipeline

training_args = TrainingArguments(
    output_dir=output_dir, 
    evaluation_strategy="steps",
    save_steps=EVALUATION_STEPS,
    max_steps=NUM_STEPS_PER_EPOCH,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_steps=EVALUATION_STEPS,
    report_to="tensorboard")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()